In [1]:
class SudokuCSP:
    """Sudoku as a Constraint Satisfaction Problem"""
    
    def __init__(self, puzzle):
        """
        Initialize Sudoku CSP.
        puzzle: 9x9 list where 0 represents empty cell
        """
        self.size = 9
        self.box_size = 3
        self.puzzle = [row[:] for row in puzzle]  # Deep copy
        
        # Variables: each cell (i,j)
        # Domains: possible values for each cell
        self.domains = {}
        for i in range(self.size):
            for j in range(self.size):
                if puzzle[i][j] == 0:
                    # Empty cell: domain is all digits 1-9
                    self.domains[(i, j)] = set(range(1, 10))
                else:
                    # Given cell: domain is fixed value
                    self.domains[(i, j)] = {puzzle[i][j]}
    
    def get_row_neighbors(self, row, col):
        """Get all cells in the same row"""
        return [(row, c) for c in range(self.size) if c != col]
    
    def get_col_neighbors(self, row, col):
        """Get all cells in the same column"""
        return [(r, col) for r in range(self.size) if r != row]
    
    def get_box_neighbors(self, row, col):
        """Get all cells in the same 3x3 box"""
        box_row = (row // self.box_size) * self.box_size
        box_col = (col // self.box_size) * self.box_size
        neighbors = []
        for i in range(box_row, box_row + self.box_size):
            for j in range(box_col, box_col + self.box_size):
                if (i, j) != (row, col):
                    neighbors.append((i, j))
        return neighbors
    
    def get_all_neighbors(self, row, col):
        """Get all cells that constrain this cell"""
        neighbors = set()
        neighbors.update(self.get_row_neighbors(row, col))
        neighbors.update(self.get_col_neighbors(row, col))
        neighbors.update(self.get_box_neighbors(row, col))
        return list(neighbors)
    
    def is_consistent(self, row, col, value):
        """
        Check if assigning value to (row, col) violates constraints.
        A value is consistent if it doesn't appear in any neighbor.
        """
        # Check row constraint
        for c in range(self.size):
            if c != col and self.puzzle[row][c] == value:
                return False
        
        # Check column constraint
        for r in range(self.size):
            if r != row and self.puzzle[r][col] == value:
                return False
        
        # Check box constraint
        box_row = (row // self.box_size) * self.box_size
        box_col = (col // self.box_size) * self.box_size
        for i in range(box_row, box_row + self.box_size):
            for j in range(box_col, box_col + self.box_size):
                if (i, j) != (row, col) and self.puzzle[i][j] == value:
                    return False
        
        return True
    
    def is_complete(self):
        """Check if all cells are assigned"""
        for i in range(self.size):
            for j in range(self.size):
                if self.puzzle[i][j] == 0:
                    return False
        return True
    
    def print_puzzle(self, title="Sudoku"):
        """Pretty print the puzzle"""
        print(f"\n{title}")
        print("─" * 25)
        for i in range(self.size):
            if i > 0 and i % 3 == 0:
                print("─" * 25)
            row_str = ""
            for j in range(self.size):
                if j > 0 and j % 3 == 0:
                    row_str += "│ "
                val = self.puzzle[i][j]
                row_str += str(val) if val != 0 else "."
                row_str += " "
            print(row_str)
        print()

# Demonstration: Easy 4x4 Sudoku (for clarity)
print("=== CSP Formulation of Sudoku ===\n")

# Simple 4x4 Sudoku for demonstration (2x2 boxes)
# This makes it easier to see all variables and constraints
simple_puzzle_4x4 = [
    [1, 0, 0, 4],
    [0, 4, 1, 0],
    [4, 0, 0, 1],
    [0, 1, 4, 0]
]

print("4x4 Sudoku (2x2 boxes):")
print("Variables: 16 cells")
print("Domains: {1, 2, 3, 4} for each cell")
print("Constraints: No repeats in rows, columns, or 2x2 boxes\n")

# Show the puzzle structure
for i, row in enumerate(simple_puzzle_4x4):
    print(f"Row {i}: {row}")

print("\n--- Constraint Analysis ---\n")

# Demonstrate constraint checking
test_cell = (0, 1)  # Second cell in first row
print(f"Cell {test_cell}: Currently empty (0)")
print(f"Row neighbors: {[(0, c) for c in range(4) if c != test_cell[1]]}")
print(f"Column neighbors: {[(r, 1) for r in range(4) if r != test_cell[0]]}")
print(f"Box neighbors: {[(r, c) for r in range(2) for c in range(2) if (r,c) != test_cell]}")

print(f"\nCurrent row values: {simple_puzzle_4x4[0]}")
print(f"Values 1 and 4 already used in row → cannot assign 1 or 4")
print(f"Therefore, domain for cell {test_cell} = {{2, 3}}")

# Standard 9x9 Sudoku (easy puzzle)
easy_puzzle = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
]

csp = SudokuCSP(easy_puzzle)
csp.print_puzzle("Easy Sudoku Puzzle")

# Show domain information
print("Domain Analysis:")
empty_cells = [(i, j) for i in range(9) for j in range(9) if easy_puzzle[i][j] == 0]
print(f"Total variables: 81")
print(f"Given values: {81 - len(empty_cells)}")
print(f"Empty cells (variables to solve): {len(empty_cells)}")
print(f"\nSample domains (first 5 empty cells):")
for cell in empty_cells[:5]:
    print(f"  Cell {cell}: domain = {sorted(csp.domains[cell])}")

=== CSP Formulation of Sudoku ===

4x4 Sudoku (2x2 boxes):
Variables: 16 cells
Domains: {1, 2, 3, 4} for each cell
Constraints: No repeats in rows, columns, or 2x2 boxes

Row 0: [1, 0, 0, 4]
Row 1: [0, 4, 1, 0]
Row 2: [4, 0, 0, 1]
Row 3: [0, 1, 4, 0]

--- Constraint Analysis ---

Cell (0, 1): Currently empty (0)
Row neighbors: [(0, 0), (0, 2), (0, 3)]
Column neighbors: [(1, 1), (2, 1), (3, 1)]
Box neighbors: [(0, 0), (1, 0), (1, 1)]

Current row values: [1, 0, 0, 4]
Values 1 and 4 already used in row → cannot assign 1 or 4
Therefore, domain for cell (0, 1) = {2, 3}

Easy Sudoku Puzzle
─────────────────────────
5 3 . │ . 7 . │ . . . 
6 . . │ 1 9 5 │ . . . 
. 9 8 │ . . . │ . 6 . 
─────────────────────────
8 . . │ . 6 . │ . . 3 
4 . . │ 8 . 3 │ . . 1 
7 . . │ . 2 . │ . . 6 
─────────────────────────
. 6 . │ . . . │ 2 8 . 
. . . │ 4 1 9 │ . . 5 
. . . │ . 8 . │ . 7 9 

Domain Analysis:
Total variables: 81
Given values: 30
Empty cells (variables to solve): 51

Sample domains (first 5 empty 

In [2]:
import time

class NaiveBacktrackingSolver:
    """Basic backtracking without any optimizations"""
    
    def __init__(self, csp):
        self.csp = csp
        self.assignments = 0  # Counter for performance analysis
        self.backtracks = 0
    
    def solve(self):
        """Find solution using naive backtracking"""
        self.assignments = 0
        self.backtracks = 0
        start_time = time.time()
        
        result = self._backtrack()
        
        elapsed = time.time() - start_time
        return result, elapsed
    
    def _backtrack(self):
        """Recursive backtracking"""
        # Check if assignment is complete
        if self.csp.is_complete():
            return True
        
        # Select first unassigned variable (naive ordering)
        var = self._select_unassigned_variable()
        if var is None:
            return True
        
        row, col = var
        
        # Try each value in domain (naive ordering: 1-9)
        for value in range(1, 10):
            self.assignments += 1
            
            # Check if value is consistent with current assignment
            if self.csp.is_consistent(row, col, value):
                # Assign value
                self.csp.puzzle[row][col] = value
                
                # Recursive call
                result = self._backtrack()
                if result:
                    return True
                
                # Undo assignment (backtrack)
                self.csp.puzzle[row][col] = 0
                self.backtracks += 1
        
        return False
    
    def _select_unassigned_variable(self):
        """Select first empty cell (naive, left-to-right, top-to-bottom)"""
        for i in range(self.csp.size):
            for j in range(self.csp.size):
                if self.csp.puzzle[i][j] == 0:
                    return (i, j)
        return None

# Test on easy puzzle
print("=== Naive Backtracking Search ===\n")

easy_puzzle = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
]

csp = SudokuCSP(easy_puzzle)
csp.print_puzzle("Initial Puzzle")

solver = NaiveBacktrackingSolver(csp)
solved, elapsed = solver.solve()

if solved:
    csp.print_puzzle("Solved Puzzle")
    print(f"Success!")
    print(f"Assignments tried: {solver.assignments}")
    print(f"Backtracks: {solver.backtracks}")
    print(f"Time: {elapsed:.4f} seconds")
else:
    print("No solution found")

print("\n--- Performance Analysis ---")
print(f"Search tree nodes explored: {solver.assignments}")
print(f"Wasted effort (backtracks): {solver.backtracks}")
print(f"Efficiency ratio: {solver.backtracks / max(1, solver.assignments):.2%} wasted")

# Compare with slightly harder puzzle
print("\n=== Testing on Medium Puzzle ===\n")

medium_puzzle = [
    [0, 0, 0, 6, 0, 0, 4, 0, 0],
    [7, 0, 0, 0, 0, 3, 6, 0, 0],
    [0, 0, 0, 0, 9, 1, 0, 8, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 5, 0, 1, 8, 0, 0, 0, 3],
    [0, 0, 0, 3, 0, 6, 0, 4, 5],
    [0, 4, 0, 2, 0, 0, 0, 6, 0],
    [9, 0, 3, 0, 0, 0, 0, 0, 0],
    [0, 2, 0, 0, 0, 0, 1, 0, 0]
]

csp2 = SudokuCSP(medium_puzzle)
solver2 = NaiveBacktrackingSolver(csp2)
solved2, elapsed2 = solver2.solve()

if solved2:
    csp.print_puzzle("Solved Puzzle")
    print(f"Success!")
    print(f"Assignments tried: {solver2.assignments}")
    print(f"Backtracks: {solver2.backtracks}")
    print(f"Time: {elapsed2:.4f} seconds")
else:
    print("No solution found")

print("\n--- Performance Analysis ---")
print(f"Search tree nodes explored: {solver2.assignments}")
print(f"Wasted effort (backtracks): {solver2.backtracks}")
print(f"Efficiency ratio: {solver2.backtracks / max(1, solver2.assignments):.2%} wasted")

=== Naive Backtracking Search ===


Initial Puzzle
─────────────────────────
5 3 . │ . 7 . │ . . . 
6 . . │ 1 9 5 │ . . . 
. 9 8 │ . . . │ . 6 . 
─────────────────────────
8 . . │ . 6 . │ . . 3 
4 . . │ 8 . 3 │ . . 1 
7 . . │ . 2 . │ . . 6 
─────────────────────────
. 6 . │ . . . │ 2 8 . 
. . . │ 4 1 9 │ . . 5 
. . . │ . 8 . │ . 7 9 


Solved Puzzle
─────────────────────────
5 3 4 │ 6 7 8 │ 9 1 2 
6 7 2 │ 1 9 5 │ 3 4 8 
1 9 8 │ 3 4 2 │ 5 6 7 
─────────────────────────
8 5 9 │ 7 6 1 │ 4 2 3 
4 2 6 │ 8 5 3 │ 7 9 1 
7 1 3 │ 9 2 4 │ 8 5 6 
─────────────────────────
9 6 1 │ 5 3 7 │ 2 8 4 
2 8 7 │ 4 1 9 │ 6 3 5 
3 4 5 │ 2 8 6 │ 1 7 9 

Success!
Assignments tried: 37652
Backtracks: 4157
Time: 0.0386 seconds

--- Performance Analysis ---
Search tree nodes explored: 37652
Wasted effort (backtracks): 4157
Efficiency ratio: 11.04% wasted

=== Testing on Medium Puzzle ===


Solved Puzzle
─────────────────────────
5 3 4 │ 6 7 8 │ 9 1 2 
6 7 2 │ 1 9 5 │ 3 4 8 
1 9 8 │ 3 4 2 │ 5 6 7 
────────────────

In [4]:
from copy import deepcopy

class ForwardCheckingSolver:
    """Backtracking with forward checking"""
    
    def __init__(self, csp):
        self.csp = csp
        self.assignments = 0
        self.backtracks = 0
        self.domain_reductions = 0
    
    def solve(self):
        """Solve using forward checking"""
        self.assignments = 0
        self.backtracks = 0
        self.domain_reductions = 0
        start_time = time.time()
        
        # Initialize domains by removing values inconsistent with givens
        self._initial_forward_check()
        
        result = self._backtrack()
        elapsed = time.time() - start_time
        return result, elapsed
    
    def _initial_forward_check(self):
        """Remove values from domains based on initial assignments"""
        for i in range(self.csp.size):
            for j in range(self.csp.size):
                if self.csp.puzzle[i][j] != 0:
                    value = self.csp.puzzle[i][j]
                    # Remove this value from all neighbors
                    for neighbor in self.csp.get_all_neighbors(i, j):
                        ni, nj = neighbor
                        if value in self.csp.domains[(ni, nj)]:
                            self.csp.domains[(ni, nj)].discard(value)
    
    def _backtrack(self):
        """Recursive backtracking with forward checking"""
        if self.csp.is_complete():
            return True
        
        # Select unassigned variable
        var = self._select_unassigned_variable()
        if var is None:
            return True
        
        row, col = var
        
        # Try each value in this variable's domain
        for value in list(self.csp.domains[(row, col)]):
            self.assignments += 1
            
            if self.csp.is_consistent(row, col, value):
                # Assign value
                self.csp.puzzle[row][col] = value
                
                # Save domains before forward checking
                saved_domains = deepcopy(self.csp.domains)
                
                # Forward check: remove value from neighbor domains
                if self._forward_check(row, col, value):
                    # No domain wipeout, continue search
                    result = self._backtrack()
                    if result:
                        return True
                
                # Restore domains and undo assignment
                self.csp.domains = saved_domains
                self.csp.puzzle[row][col] = 0
                self.backtracks += 1
        
        return False
    
    def _forward_check(self, row, col, value):
        """
        Remove value from all neighbor domains.
        Returns False if any domain becomes empty (wipeout).
        """
        for neighbor in self.csp.get_all_neighbors(row, col):
            ni, nj = neighbor
            if self.csp.puzzle[ni][nj] == 0:  # Unassigned
                if value in self.csp.domains[(ni, nj)]:
                    self.csp.domains[(ni, nj)].discard(value)
                    self.domain_reductions += 1
                    
                    # Check for domain wipeout
                    if len(self.csp.domains[(ni, nj)]) == 0:
                        return False
        
        return True
    
    def _select_unassigned_variable(self):
        """Select first unassigned variable (naive ordering)"""
        for i in range(self.csp.size):
            for j in range(self.csp.size):
                if self.csp.puzzle[i][j] == 0:
                    return (i, j)
        return None

# Test forward checking
print("=== Forward Checking ===\n")

easy_puzzle = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
]

csp = SudokuCSP(easy_puzzle)
print("Domains after initialization:")
empty_cells = [(i, j) for i in range(9) for j in range(9) if easy_puzzle[i][j] == 0]
print(f"Sample cell (0,2): domain = {sorted(csp.domains[(0, 2)])}")

solver = ForwardCheckingSolver(csp)
solved, elapsed = solver.solve()

if solved:
    csp.print_puzzle("Solved with Forward Checking")
    print(f"Assignments: {solver.assignments}")
    print(f"Backtracks: {solver.backtracks}")
    print(f"Domain reductions: {solver.domain_reductions}")
    print(f"Time: {elapsed:.4f} seconds")

# Compare with naive backtracking
print("\n--- Comparison: Forward Checking vs Naive Backtracking ---\n")

csp_naive = SudokuCSP(easy_puzzle)
solver_naive = NaiveBacktrackingSolver(csp_naive)
_, time_naive = solver_naive.solve()

print(f"{'Metric':<25} {'Naive':<15} {'Forward Checking':<15} {'Improvement'}")
print("─" * 70)
print(f"{'Assignments':<25} {solver_naive.assignments:<15} {solver.assignments:<15} {solver_naive.assignments/max(1,solver.assignments):.1f}x fewer")
print(f"{'Time (seconds)':<25} {time_naive:<15.4f} {elapsed:<15.4f} {time_naive/max(0.0001,elapsed):.1f}x faster")
print(f"{'Backtracks':<25} {solver_naive.backtracks:<15} {solver.backtracks:<15} {solver_naive.backtracks/max(1,solver.backtracks):.1f}x fewer")

=== Forward Checking ===

Domains after initialization:
Sample cell (0,2): domain = [1, 2, 3, 4, 5, 6, 7, 8, 9]

Solved with Forward Checking
─────────────────────────
5 3 4 │ 6 7 8 │ 9 1 2 
6 7 2 │ 1 9 5 │ 3 4 8 
1 9 8 │ 3 4 2 │ 5 6 7 
─────────────────────────
8 5 9 │ 7 6 1 │ 4 2 3 
4 2 6 │ 8 5 3 │ 7 9 1 
7 1 3 │ 9 2 4 │ 8 5 6 
─────────────────────────
9 6 1 │ 5 3 7 │ 2 8 4 
2 8 7 │ 4 1 9 │ 6 3 5 
3 4 5 │ 2 8 6 │ 1 7 9 

Assignments: 490
Backtracks: 439
Domain reductions: 1480
Time: 0.1423 seconds

--- Comparison: Forward Checking vs Naive Backtracking ---

Metric                    Naive           Forward Checking Improvement
──────────────────────────────────────────────────────────────────────
Assignments               37652           490             76.8x fewer
Time (seconds)            0.0341          0.1423          0.2x faster
Backtracks                4157            439             9.5x fewer
